왜 contrastive learning이 generative, predictive learning보다 더 좋은 representation을 학습하는가? 

-> loss를 정의하는 space가 다름... 음 좀 더 자세히... 





In [1]:
import numpy as np
import torch
from torch import nn
from torchvision.transforms import transforms

In [2]:
import argparse 

args = argparse.Namespace()

args.data='./datasets' 
args.dataset_name='cifar10'
args.n_views = 2
args.workers=1
args.arch='resnet50'
args.out_dim=128
args.wd=0.0005
args.batch_size=256
args.device='cuda'
args.temperature=0.07

In [3]:
import torch.nn.functional as F

In [4]:
features = torch.rand(2*args.batch_size,2048) # mimic ResNet50 embedding vector

In [5]:
features[0].std()

tensor(0.2861)

In [6]:
labels = torch.cat([torch.arange(args.batch_size) for i in range(args.n_views)], dim=0) 
# 2 x batch size, simply index array x2
print(labels)

tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
        154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
        168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 1

In [7]:
labels = (labels.unsqueeze(0) == labels.unsqueeze(1)).float()
print(labels)

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])


In [8]:
labels = labels.to(args.device)

features = F.normalize(features, dim=1)

In [9]:
features[0].max()

tensor(0.0385)

In [10]:
torch.sum(features, axis=0)

tensor([ 9.5212, 10.0602,  9.7840,  ...,  9.7557, 10.0513,  9.5111])

In [11]:
similarity_matrix = torch.matmul(features, features.T)
# assert similarity_matrix.shape == (
#     args.n_views * args.batch_size, args.n_views * args.batch_size)
# assert similarity_matrix.shape == labels.shape

# discard the main diagonal from both: labels and similarities matrix
mask = torch.eye(labels.shape[0], dtype=torch.bool).to(args.device)

In [16]:
labels = labels[~mask].view(labels.shape[0], -1)

In [18]:
labels.shape

torch.Size([512, 511])

In [19]:
similarity_matrix

tensor([[1.0000, 0.7547, 0.7486,  ..., 0.7443, 0.7488, 0.7517],
        [0.7547, 1.0000, 0.7499,  ..., 0.7497, 0.7597, 0.7498],
        [0.7486, 0.7499, 1.0000,  ..., 0.7553, 0.7439, 0.7462],
        ...,
        [0.7443, 0.7497, 0.7553,  ..., 1.0000, 0.7463, 0.7432],
        [0.7488, 0.7597, 0.7439,  ..., 0.7463, 1.0000, 0.7543],
        [0.7517, 0.7498, 0.7462,  ..., 0.7432, 0.7543, 1.0000]])

In [20]:
similarity_matrix = similarity_matrix[~mask].view(similarity_matrix.shape[0], -1)
# assert similarity_matrix.shape == labels.shape

In [25]:
# select and combine multiple positives
positives = similarity_matrix[labels.bool()].view(labels.shape[0], -1)

# select only the negatives the negatives
negatives = similarity_matrix[~labels.bool()].view(similarity_matrix.shape[0], -1)

logits = torch.cat([positives, negatives], dim=1)
labels = torch.zeros(logits.shape[0], dtype=torch.long).to(args.device)

logits = logits / args.temperature

torch.Size([512, 512])


In [22]:
logits.shape

torch.Size([512, 511])